In [1]:
import pandas as pd
from docplex.mp.model import Model
mdl=Model('SMP')
df = pd.read_excel('Tabla Instancia.xlsx',sheet_name='Cij', index_col='Cij')


In [2]:
def time(seq):
    timecheck = 0
    timer = 0
    for i in range(len(seq)):
        if i == 0:
            timecheck = 0
        else:
            timer += df.iloc[seq[i-1],seq[i]]
            timecheck += df.iloc[seq[i-1],seq[i]]+timer
    return timecheck-timer

In [3]:
#machines
n = len(df)-1
m = 4
N = n - m + 1
print(n)
print(m)
print(N)

15
4
12


In [4]:
arcos =[(i,j,k) for i in range (0, n+1) for j in range (0,n+1) if i!=j  for k in range (0, N+2)]
x=mdl.binary_var_dict(arcos,name='x')

### Objective function

# $Min \; Z= \; \displaystyle\sum_{j=1}^{n}C_{0j}\sum_{k=1}^{N} Ky^{k}_{0j}+\sum_{i=1}^{n}\sum_{j=1, j\neq i}^{n}C_{ij}\sum_{k=1}^{N} Ky^{k}_{ij}$

In [5]:
mdl.minimize(mdl.sum(df.iloc[0][j] * mdl.sum(k * x[0,j,k] for k in range(1,N+1)) for j in range(1,n+1)) + mdl.sum(df.iloc[i][j] *mdl.sum(k * x[i,j,k] for k in range(1,N)) for j in range(1,n+1) for i in range(1,n+1) if j!=i))

# $\displaystyle\sum_{k=1}^{N} y^{k}_{0j}+\sum_{k=1}^{N-1}\sum_{i=1, i\neq j}^{n} y^{k}_{ij} = 1$     $\; \; (\forall \;j \; \in \; I)$

# $\displaystyle\sum_{j=1}^{n} y^{1}_{0j}+\sum_{i=1}^{n}\sum_{j=1, j\neq i}^{n} y^{1}_{ij} = m$     $\; \; $

# $\displaystyle\sum_{k=1}^{N} \sum_{j=1}^{n} y^{k}_{0j}=m $

# $\displaystyle y^{k+1}_{0j} +  \sum_{i=1, i\neq j}^{n} y^{k+1}_{ij} = \sum_{i=1, i\neq j}^{n} y^{k}_{ji} $     $\; \; (\forall \;j \; \in \; I ; k=1,2,...,N-2)$

# $\displaystyle y^{N}_{0j} = \sum_{i=1, i\neq j}^{n} y^{N-1}_{ji} $     $\; \; (\forall \;j \; \in \; I)$

In [6]:
# Restricciones:

#1
for j in range(1,n+1):
    mdl.add_constraint((mdl.sum(x[0,j,k] for k in range(1,N+1))+mdl.sum(x[i, j, k] for k in range(1,N) for i in range(1,n+1) if i!=j)) ==1)
                      
#2
    mdl.add_constraint((mdl.sum(x[0,j,1] for j in range(1,n+1)) + mdl.sum(x[i,j,1] for j in range(1,n+1) for i in range(1,n+1) if j!=i)) == m)
                        
#3
    mdl.add_constraint((mdl.sum(x[0,j,k] for k in range(1,N+1) for j in range(1,n+1))) == m)
#4
for k in range(1,N-1):
    for j in range(1,n+1):
        mdl.add_constraint((x[0,j,k+1] + mdl.sum(x[i,j,k+1] for i in range(1,n+1) if i!=j)) == mdl.sum(x[j,i,k] for i in range(1,n+1) if i!=j))
#5    
for j in range(1,n+1):
    mdl.add_constraint((x[0,j,N]) == mdl.sum(x[j,i,N-1] for i in range(1,n+1) if i!=j))

In [7]:
#mdl.parameters.timelimit=120
#mdl.parameters.mip.strategy.branch=1
#mdl.parameters.mip.tolerances.mipgap=0.15

solucion = mdl.solve(log_output=True)


Version identifier: 12.10.0.0 | 2019-11-26 | 843d4de2ae
CPXPARAM_Read_DataCheck                          1
CPXPARAM_RandomSeed                              201903125
Tried aggregator 1 time.
MIP Presolve eliminated 28 rows and 870 columns.
Reduced MIP has 182 rows, 2490 columns, and 7470 nonzeros.
Reduced MIP has 2490 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (5.55 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 182 rows, 2490 columns, and 7470 nonzeros.
Reduced MIP has 2490 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (4.60 ticks)
Probing time = 0.00 sec. (4.79 ticks)
Clique table members: 5122.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 12 threads.
Root relaxation solution time = 0.02 sec. (8.77 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best 

In [8]:
mdl.get_solve_status()

<JobSolveStatus.OPTIMAL_SOLUTION: 2>

In [9]:
solucion.display()

solution for: SMP
objective: 2539
x_0_4_3 = 1
x_0_9_4 = 1
x_0_11_3 = 1
x_0_15_5 = 1
x_1_6_1 = 1
x_2_1_2 = 1
x_4_7_2 = 1
x_5_3_1 = 1
x_7_13_1 = 1
x_8_10_1 = 1
x_9_12_3 = 1
x_11_5_2 = 1
x_12_8_2 = 1
x_14_2_3 = 1
x_15_14_4 = 1


In [10]:
arcos_activos = [arco for arco in arcos if x[arco].solution_value>0.9]
print(arcos_activos)

[(0, 4, 3), (0, 9, 4), (0, 11, 3), (0, 15, 5), (1, 6, 1), (2, 1, 2), (4, 7, 2), (5, 3, 1), (7, 13, 1), (8, 10, 1), (9, 12, 3), (11, 5, 2), (12, 8, 2), (14, 2, 3), (15, 14, 4)]


In [11]:
pd.DataFrame(arcos_activos)

,0,1,2
0,0,4,3
1,0,9,4
2,0,11,3
3,0,15,5
4,1,6,1
5,2,1,2
6,4,7,2
7,5,3,1
8,7,13,1
9,8,10,1


In [12]:
mseq = {}
mseq[1] = {'TCT': 0 , 'Sequence': [0,4,7,13]}
mseq[2] = {'TCT': 0 , 'Sequence': [0,9,12,8,10]}
mseq[3] = {'TCT': 0 , 'Sequence': [0,11,5,3]}
mseq[4] = {'TCT': 0 , 'Sequence': [0,15,14,2,1,6]}

In [13]:
total = 0
for i in range(1, m+1):
    mseq[i]['TCT'] = time(mseq[i]['Sequence'])
    total += mseq[i]['TCT']
mseq

{1: {'TCT': 517, 'Sequence': [0, 4, 7, 13]},
 2: {'TCT': 636, 'Sequence': [0, 9, 12, 8, 10]},
 3: {'TCT': 647, 'Sequence': [0, 11, 5, 3]},
 4: {'TCT': 739, 'Sequence': [0, 15, 14, 2, 1, 6]}}

In [14]:
total

2539